# Introduction to Neural Network (Digits Recognition) Model [Tutorial]

In [1]:
# image data
from sklearn.datasets import load_digits

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# data processing
import pandas as pd

In [2]:
digits = load_digits()

### Prepare data for modeling

In [3]:
# grab all data (1797 records, and 8x8=64 columns)
X = digits.data

# grab the target (true) value for each image
y = digits.target

X_train, X_test, y_train_raw, y_test_raw = train_test_split(X, y, random_state=314)

y_train = pd.get_dummies(y_train_raw).values
y_test = pd.get_dummies(y_test_raw).values

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1347, 64) (1347, 10) (450, 64) (450, 10)


Before we start, we need to take a small digression. The keras Neural Network model training results are not easily reproducible since it involves a lot of shuffling and random initializations. In order to maintain consitency, we will have to initialize some random seeds before every model run. We will create a function to do this.

In [4]:
from tensorflow import random as tf_random
import numpy as np
import random

def init_seeds(s):
    '''
    Initializes random seeds prior to model training 
    to ensure reproducibality of training results.
    '''
    tf_random.set_seed(s)
    np.random.seed(s)
    random.seed(s)

### Multi-layer Perceptron (MLP) model

Define a model with one input layer and one output layer.

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

mlp1 = Sequential(
    [
        Input(shape=(64,)),
        Dense(10, activation='softmax')
    ]
)

In [24]:
mlp1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

We have initialized a sequential model with an input shape of 64, and an output layer with a shape of 10. There are 650 total parameters in this model:

In [6]:
# initial weights and biases
#--

_**Useful Resources:**_

* [Keras Documentation: The Sequential model](https://keras.io/guides/sequential_model/)
* [TensorFlow API Documentation: tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)

Before we can train the model, we need to specify training parameters (aka compile the model).

In [25]:
mlp1.compile(optimizer='sgd', loss='categorical_crossentropy',
             metrics=['accuracy'])

#### Train (fit) the model

In [26]:
mlp1.fit(X_train, y_train, shuffle=False) #should be set to true

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.3024 - loss: 6.1153  


The accuracy is very low, because the model made a single pass over the dataset. The model must make multiple passes over the entire dataset in order to produce better results. This can be adjusted by using the `epoch` parameter.

Epoch is equal to the number of times the algorithm sees the entire dataset.

By the way, what is that mysterious-looking number 43? Where did that come from?

Keras uses a `batch size` of 32 by default. So this model above created 32 batches of 43 records each by divinding the total number of records in the entire dataset (1,347) by 32. However, the default `epoch` is 1, so the model above made 43 **iterations** (forward + backward) using batches of 52 records.

`batch_size * number of iterations` --> `epoch`

Note that if you use `shuffle=True` in the `fit()` function, keras will shuffle the records in the training dataset before splitting them into batches.

Let's increase the number of epochs.

In [32]:
# initialize seeds
init_seeds(314)

# prepare the model architecture
mlp2 = Sequential(
    [
        Input(shape=(64,)),
        Dense(10, activation='softmax', name='output_layer')
    ],
    name='simple_mlp_7_epochs')
mlp2.compile(optimizer='sgd', loss='categorical_crossentropy',
             metrics=['accuracy'])
mlp2.fit(X_train, y_train, epochs=7, shuffle=True)

Epoch 1/7
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.3206 - loss: 6.2504  
Epoch 2/7
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step - accuracy: 0.7463 - loss: 1.2794
Epoch 3/7
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step - accuracy: 0.8264 - loss: 0.6766
Epoch 4/7
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9152 - loss: 0.3117
Epoch 5/7
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 629us/step - accuracy: 0.9299 - loss: 0.2604
Epoch 6/7
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.9378 - loss: 0.2259
Epoch 7/7
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - accuracy: 0.9435 - loss: 0.1999


Note that now the model is making 7 passes through the entire data. For each epoch, there are 43 iterations (forward + backward) with 32 samples in each iteration.

Let's try to reduce the batch size and see what happens.

In [35]:
# initialize seeds
init_seeds(314)

# prepare the model architecture
mlp3 = Sequential(
    [
        Input(shape=(64,)),
        Dense(10, activation='softmax', name='output_layer')
    ], 
    name='simple_mlp_7epochs_5bs')

mlp3.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

mlp3.fit(X_train, y_train, batch_size=5, epochs=7, shuffle=True);

Epoch 1/7
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - accuracy: 0.5935 - loss: 3.6357   
Epoch 2/7
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - accuracy: 0.9097 - loss: 0.3841
Epoch 3/7
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - accuracy: 0.9492 - loss: 0.2488
Epoch 4/7
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.9513 - loss: 0.1799
Epoch 5/7
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - accuracy: 0.9712 - loss: 0.1412
Epoch 6/7
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - accuracy: 0.9727 - loss: 0.1111
Epoch 7/7
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - accuracy: 0.9745 - loss: 0.0987


The batch size of 5 gives us 270 iterations for each epoch.

**Useful Resources:**
* [Keras FAQ: What do sample, batch, and epoch mean?](https://keras.io/getting_started/faq/#what-do-sample-batch-and-epoch-mean)
* [What is the trade-off between batch size and number of iterations to train a neural network?](https://stats.stackexchange.com/questions/164876/what-is-the-trade-off-between-batch-size-and-number-of-iterations-to-train-a-neu)

Tip: Using larger batch sizes will require more memory, and negatively impact the ability of the model to generalize well. On the other hand, very small batch size increases the risk of making the model unreliable (too stochastic.)

#### Check the model accuracy on the test sample

In [37]:
loss, accuracy = mlp1.evaluate(X_test, y_test)

print(f'Loss: {loss:.2%}, Accuracy: {accuracy:.2%}')

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.6643 - loss: 2.4530
Loss: 265.75%, Accuracy: 63.11%


We will use this multiple times, so let's create a small function for this.

In [36]:
def test_model(model, test_data, lables):
    loss, accuracy = model.evaluate(test_data, lables, batch_size=1)
    print(f'Loss: {loss:.2%}, Accuracy: {accuracy:.2%}')
    
test_model(mlp1, X_test, y_test)

450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.6778 - loss: 2.3438
Loss: 280.43%, Accuracy: 63.11%


The model `mlp3` with a batch size of five and seven epochs seems to be performing the best.

Next, let's add one hidden layer to this model and see how much improvement it yields.

#### Add a hidden layer to the model

In [39]:
# initialize seeds
init_seeds(314)

# prepare the model architecture
mlp4 = Sequential(
    [
    Input(shape=(64,)),
    Dense(32, activation='softmax', name='hidden_layer'),
    Dense(10, activation='softmax', name='output_layer')
    ],
    name='simple_mlp_7_epochs'
)

mlp4.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

mlp4.fit(X_train, y_train, batch_size=5, epochs=15, shuffle=True, verbose=2);

Epoch 1/15


270/270 - 0s - 2ms/step - accuracy: 0.1203 - loss: 2.2696
Epoch 2/15
270/270 - 0s - 698us/step - accuracy: 0.2754 - loss: 2.1430
Epoch 3/15
270/270 - 0s - 688us/step - accuracy: 0.3185 - loss: 2.0522
Epoch 4/15
270/270 - 0s - 718us/step - accuracy: 0.3422 - loss: 1.9636
Epoch 5/15
270/270 - 0s - 839us/step - accuracy: 0.3586 - loss: 1.8837
Epoch 6/15
270/270 - 0s - 720us/step - accuracy: 0.3801 - loss: 1.8088
Epoch 7/15
270/270 - 0s - 739us/step - accuracy: 0.3898 - loss: 1.7433
Epoch 8/15
270/270 - 0s - 717us/step - accuracy: 0.3808 - loss: 1.6864
Epoch 9/15
270/270 - 0s - 694us/step - accuracy: 0.3831 - loss: 1.6322
Epoch 10/15
270/270 - 0s - 736us/step - accuracy: 0.3927 - loss: 1.5732
Epoch 11/15
270/270 - 0s - 717us/step - accuracy: 0.3890 - loss: 1.5218
Epoch 12/15
270/270 - 0s - 776us/step - accuracy: 0.3920 - loss: 1.4711
Epoch 13/15
270/270 - 0s - 720us/step - accuracy: 0.3883 - loss: 1.4279
Epoch 14/15
270/270 - 0s - 698us/step - accuracy: 0.3890 - loss: 1.3926
Epoch 15/15
27

In [40]:
test_model(mlp4, X_test, y_test)

450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 636us/step - accuracy: 0.3862 - loss: 1.3903
Loss: 141.50%, Accuracy: 39.33%


Adding one hidden layer, and increasing the number of epochs, improved the model's accuracy.

**Interesting questions:**

* [How to choose the number of hidden layers and the size of hidden layers?](https://stackoverflow.com/questions/10565868/multi-layer-perceptron-mlp-architecture-criteria-for-choosing-number-of-hidde?lq=1)
* [Why are neural networks becoming deeper, but not wider?](https://stats.stackexchange.com/questions/222883/why-are-neural-networks-becoming-deeper-but-not-wider)

#### Try a different activation function (relu)

In [44]:
# initialize seeds
init_seeds(314)

# prepare the model architecture
mlp5 = Sequential(
    [
        Input(shape=(64,)),
        Dense(32, activation='softmax', name='hidden_layer'),
        Dense(10, activation='softmax', name='output_layer')
    ],
    name='mlp_1hidden_relu')

mlp5.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

mlp5.fit(X_train, y_train, batch_size=5, epochs=15, shuffle=True, verbose=0)

test_model(mlp5, X_test, y_test)

450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - accuracy: 0.3862 - loss: 1.3903
Loss: 141.50%, Accuracy: 39.33%


Changing the activation function did not improve the model performance for this dataset. However, the `relu` activation function usually outperforms `sigmoid` and it's usually preferred.

#### Try a different optimizer (adam)

In [45]:
# initialize seeds
init_seeds(314)

# prepare the model architecture
mlp6 = Sequential(
    [
        Input(shape=8*8),
        Dense(32, activation='relu', name='hidden_layer'),
        Dense(10, activation='softmax', name='output_layer')
    ],
    name='mlp_1hidden_adam')

mlp6.compile(optimizer=, loss='categorical_crossentropy', metrics='accuracy')

mlp6.fit(X_train, y_train, batch_size=5, epochs=15, shuffle=True, verbose=0)

test_model(mlp6, X_test, y_test)

SyntaxError: expected argument value expression (4184832528.py, line 13)

The adam optimizer improved the model's performance.

**Useful resource:**

* [Keras Documentation: Adam](https://keras.io/api/optimizers/adam/)

### Plot confusion matrix

In [ ]:
# calculate predicted probabilities
probs = 

In [ ]:
# we can use formatted string literal to print float values instead
[f'{x:.5f}' for x in probs[0]]

The model returns a probability for each lable (digit). We can grab the lable (digit) that has the highest probability. For example, for the first sample, the model is predicting a very high probability of 0.99974 for that digit to be 1.

In [ ]:
# grab the digit with max probability
y_preds = 

In [ ]:
cfm = 
plt.figure(figsize=(6, 6))

sns.heatmap()
plt.xlabel('Predicted value', fontsize=12)
plt.ylabel('True value', fontsize=12)
plt.title('Confusion Matrix (Neural Network)', fontsize=12, weight='semibold');

#### Saving (exporting) the model

Save the model as a single `HDF5` file. [`HDF5` stands for Hierarchical Data Format, v5.]

In [ ]:
mlp_path = '../misc/digits_recognition_mlp_model.h5'

#--

This saves the trained model and all trackable objects (config, weights, and optimizer) attached to the model.

### Load a saved model

We can load a pre-trained (and saved) model by using `load_model()` function.

In [ ]:
#--

In [ ]:
# use the model that we just loaded to predict values
#--

**Useful resources:**
* [Keras: Serialization and saving](https://keras.io/guides/serialization_and_saving/)
* [Keras: Model saving and serialization APIs](https://keras.io/api/models/model_saving_apis/)

#### Visualize the predictions

In [ ]:
# grab raw model predictions for the entire dataset
preds_raw = 

In [ ]:
# grab the predicted digit (based on the highest probability) for the entire dataset
preds = 

Let's plot the first 100 predictions. The labels are shown in green if the model predicted the digit correcly, otherwise it's shown in red.

In [ ]:
fig, axes = plt.subplots(10, 10, figsize=(7, 7), subplot_kw={'xticks':[], 'yticks':[]})

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='binary', interpolation='bicubic')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    pred = preds[i]
    act = y[i]
    if pred == act:
        ax.text(0.05, 0.05, preds[i], color='green',
                weight='semibold', transform=ax.transAxes)
    else:
        ax.text(0.05, 0.05, preds[i], color='tomato',
                weight='semibold', transform=ax.transAxes)
plt.show();